<a href="https://colab.research.google.com/github/LuanPCunha/TCC/blob/main/Treinamento_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Configurações Iniciais 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from numpy import array
from numpy import asarray
from numpy import zeros
import string, re
import itertools
import nltk
import plotly.offline as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.callbacks import EarlyStopping
py.init_notebook_mode(connected=True)
%matplotlib inline
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

## Lendo a Base

In [9]:
# Base1 Com StopWords
PATH_BASE1 = "/content/drive/MyDrive/TCC/dados/processadas/Base1_classificada.csv"

# Base2 Sem StopWords
PATH_BASE2 = "/content/drive/MyDrive/TCC/dados/processadas/Base2_classificada.csv"

# Base1 Com StopWords
PATH_BASE1_JUNTO_COM_DA_LEILA_BALANCEADA = "/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base1JuntaHateENotHateDaLeila_balanceada.csv"

# Base2 Sem StopWords
PATH_BASE2_LEILA_LIMPISSIMA_BALANCEADA = "/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base2_maior4_menor25_limpissima_balanceada.csv"

In [10]:
#trainDF = pd.read_csv(PATH_BASE2_LEILA_LIMPISSIMA_BALANCEADA, index_col=0)
trainDF = pd.read_csv(PATH_BASE2_LEILA_LIMPISSIMA_BALANCEADA, index_col=0)

In [11]:
# Agrupa dados por label
trainDF.groupby('label').count()

,text
label,
0,2443
1,2443


In [12]:
# Pega a média de caracteres dos tweets de toda a base
average_len = int(trainDF['text'].apply(lambda x: len(str(x).split(' '))).mean())
average_len

10

In [13]:
# Pega a média de caracteres dos tweets de toda a base
max_len = int(trainDF['text'].apply(lambda x: len(str(x).split(' '))).max())
max_len

25

In [14]:
palavras_unicas = list ( trainDF['text'].str.split(' ', expand=True).stack().unique() ) 
quantidade_palavras_unicas = len( palavras_unicas )
quantidade_palavras_unicas

9144

#TF-IDF (NÃO CONSEGUIMOS USAR NO MODELO CNN)

In [54]:
# word level tf-idf
# td-idf gera matriz com 9140 colunas (4 palavras estão sendo perdidas ao usar o regex \w{1,})
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features= quantidade_palavras_unicas)
X = tfidf_vect.fit_transform(trainDF['text'])

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(X, trainDF['label'], test_size=0.3, random_state=2)

train_x, valid_x = train_x.toarray(), valid_x.toarray()
print(type(train_x))
print(valid_x)

<class 'numpy.ndarray'>
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


#LSTM

In [55]:
def train_model(classifier, feature_vector_train, label):
    # fit the training dataset on the classifier
    classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(classifier.summary())
    classifier.fit(feature_vector_train, label, epochs=3)
    return classifier

def create_rnn_lstm(input_size):
  
  model_input = layers.Input(shape=(input_size,1, ),name="input",dtype='int32')
  #embedding = layers.Embedding(len(word_index) + 1, 50, weights=[embedding_matrix])(model_input)
  embedding = layers.SpatialDropout1D(0.3)(model_input)
  lstm = layers.LSTM(1000, dropout =0.1, name="LSTM")(model_input)
  output = layers.Dense(1, activation='sigmoid',name='sigmoid')(lstm)
  model = models.Model(inputs=model_input,outputs=output)
  return model

classifier = create_rnn_lstm(max_len)
trainedModel = train_model(classifier, train_x, train_y)
# predict the labels on validation dataset
predictions = trainedModel.predict(valid_x)
predictions = predictions.round()

print("RNN-LSTM, Word Embeddings",  metrics.accuracy_score(valid_y,predictions))
print(predictions)

TypeError: ignored